# Hello, welcome to PCWO.

In this module, we will walk through running (and creating) a PCWO script. 

We are going to initialize our mp and ctl files, define the set of PhCW variables to optimize over, and then finally intiailize and run SPEA over these parameters.

First off, we have to import a few programs that are available in the PCWO backend folder.

In [1]:
from backend import constraints 
from backend.experiment import W1Experiment
from backend.spea_optimizer import SpeaOptimizer
from backend.photonicCrystalDesign import PhCWDesign
from backend.paretoFunctions import ParetoMaxFunction

Next, we record the paths to an mpb installation, a Waveguide File, and an output for our mpb computations which will be parsed.
	Note, this output file will be reused every time our algorithms evaluate a PhC design.

In [2]:
# absolute path to the mpb executable
mpb = "/Users/sean/documents/mpb-1.5/mpb/mpb"
# mpb = "mpb" # depending on how mpb is installed, the assignment "mpb" may be all that is required
# This was the case on an Ubuntu dist, I was testing on

# absolute path to the mpb waveguide input ctl
inputFile = "/Users/sean/UniversityOfOttawa/Photonics/PCWO/W1_2D_v04.ctl.txt"

# absolute path to the mpb output file
outputFile = "/Users/sean/UniversityOfOttawa/Photonics/PCWO/optimizerTestFile.txt"

We automate our mpb executions (usually done with terminal) by wrapping them in an object called an Experiment. 

The various PCWO algorithms use these experiment objects to simulate a given instance of a Photonic Crystal Design.

In [3]:
# we define a general experiment object
# that we reuse whenever we need to make a command-line mpb call
# see experiment.py for functionality
experiment = W1Experiment(mpb, inputFile, outputFile)
# ex.setParams(paramVector)
experiment.setCalculationType('4') # accepts an int from 0 to 5
experiment.setBand(23)

Now, we initialize our parameter map, load a set of predefined constraints from constraints.py in the backend folder, and store these values in a PhCWDesign object, with a score initialized to 0. Note, 'score' is not immediately useful for Optimization routines that use Pareto Functions.

In [4]:
paramMap = {}
paramMap["s1"] = 0 # First row vertical shift
paramMap["s2"] = 0 # Second row vertical shift
paramMap["s3"] = 0 # Third row vertical shift
#paramMap["p1"] = 0 # First row horizontal shift
#paramMap["p2"] = 0 # Second row horizontal shift
#paramMap["p3"] = 0 # Third row horizontal shift
paramMap["r0"] = 0.3 # Default air-hole radius
paramMap["r1"] = 0.3 # Default first row radius
paramMap["r2"] = 0.3 # Default second row radius
paramMap["r3"] = 0.3 # Default third row radius

# see constraints.py
constraintFunctions = [constraints.latticeConstraintsLD]

pcw = PhCWDesign(paramMap, 0, constraintFunctions)

SPEA requires a pareto function, in order to compare pcw designs. Because we are going to use a pareto function in order to compare PhCW, we must establish whether we want to minimize or maximize objectives. We do this by using a key map. Similarily to in paramMap, only the terms that are added to this dictionary will be optimized for. 

We use a ParetoMaxFunction by convention. This function will invert objectives in the key map that have been specified as "min". In other words, if key_map["loss"] = "min", then 1/loss will be evaluated and maximized.


In [5]:
#Initialize pareto function

key_map = {}
key_map["ng0"] = "max"
key_map["loss_at_ng0"] = "min"

pareto_function = ParetoMaxFunction(experiment, key_map)

Now, we define the parameters for SPEA, because each PhCW takes a long time to simulate, one should be conservative with the population size and number of generations

In [6]:
#SPEA parameters

max_generation = 10 # number of iterations of the SPEA algorithm
population_size = 10 # number of solutions to consider 
pareto_archive_size = 8 # number of solutions to store after each generation
tournament_selection_rate  = 5 # number of solutions to consider in crossover/mutation

Finally, we will run our SPEA optimizer, and output results

In [ ]:
population = SpeaOptimizer.createPopulation(population_size, pcw)

optimizer = SpeaOptimizer(pareto_function)

optimizer.optimize(population,max_generation,tournament_selection_rate, pareto_archive_size)



Generation 0
DEBUG: Solution: {'r0': 0.270008, 'r1': 0.205227, 'r2': 0.256506, 'r3': 0.251328, 's3': -0.000852, 's2': 0.054554, 's1': -0.000944}
DEBUG: Loss Map: {1: 2.076299648552558, 2: 2.2798491892229853, 3: 2.520919933996712, 4: 2.8169831980313917, 5: 3.176913366219006, 6: 3.63096123685643, 7: 4.207207620789227, 8: 4.937056992832557, 9: 5.896219447009731, 10: 7.130777042013955, 11: 8.681537474977759, 12: 10.600152794474722, 13: 12.783482819654653, 14: 15.04419056340236, 15: 16.978180911591572, 16: 18.414626062450395, 17: 18.936034516402493, 18: 18.56572176581745, 19: 17.61048578762895, 20: 16.75273709823712, 21: 17.32158514826194, 22: 20.70037902461481, 23: 29.49464686229156, 24: 45.89185085122796, 25: 73.80402865290725, 26: 118.73895860083422, 27: 189.41809803704172, 28: 296.4021104173179, 29: 456.9949288354591, 30: 701.1656604538191, 31: 1065.7297349261523, 32: 1622.268605910326, 33: 2551.740635012967, 34: 3877.980587867238, 35: 6017.417613556365, 36: 9933.57858598761, 37: 1815